In [ ]:
import cv2
import numpy as np

In [ ]:
from itertools import combinations_with_replacement
from collections import defaultdict

import numpy as np
from numpy.linalg import inv

 These filters are applied to the input hazy image to obtain two important components: the bright channel and the dark channel. The bright channel represents the maximum intensity value in a local neighborhood, and the dark channel represents the minimum intensity value. These channels help identify the presence of haze in the image.


In [ ]:
def get_illumination_channel(I, w):
    M, N, _ = I.shape
    # padding for channels
    padded = np.pad(I, ((int(w/2), int(w/2)), (int(w/2), int(w/2)), (0, 0)), 'edge')
    darkch = np.zeros((M, N))
    brightch = np.zeros((M, N))

    for i, j in np.ndindex(darkch.shape):
        darkch[i, j] = np.min(padded[i:i + w, j:j + w, :]) # dark channel
        brightch[i, j] = np.max(padded[i:i + w, j:j + w, :]) # bright channel

    return darkch, brightch

The global atmosphere light is calculated based on the bright channel. It is typically computed as the maximum value in the bright channel and represents the overall illumination in the scene. The atmosphere light is a key parameter used in estimating the transmission of light through the haze.


In [ ]:
def get_atmosphere(I, brightch, p=0.1):
    M, N = brightch.shape
    flatI = I.reshape(M*N, 3) # reshaping image array
    flatbright = brightch.ravel() #flattening image array

    searchidx = (-flatbright).argsort()[:int(M*N*p)] # sorting and slicing
    A = np.mean(flatI.take(searchidx, axis=0), dtype=np.float64, axis=0)
    return A

The bright channel is used to estimate an initial transmission map. The transmission map is a value between 0 and 1 at each pixel, indicating the fraction of light that has not been scattered or absorbed by the haze. This step provides a rough estimate of how much haze is present in different parts of the image.


In [ ]:
def get_initial_transmission(A, brightch):
    A_c = np.max(A)
    init_t = (brightch-A_c)/(1.-A_c) # finding initial transmission map
    return (init_t - np.min(init_t))/(np.max(init_t) - np.min(init_t))

The dark channel is used to correct potentially erroneous transmission estimations from the previous step. Since the dark channel highlights the presence of haze, it can help refine the transmission map by identifying areas that may not have been accurately estimated using the bright channel alone.


In [ ]:
def get_corrected_transmission(I, A, darkch, brightch, init_t, alpha, omega, w):
    im = np.empty(I.shape, I.dtype);
    for ind in range(0, 3):
        im[:, :, ind] = I[:, :, ind] / A[ind] #divide pixel values by atmospheric light
    dark_c, _ = get_illumination_channel(im, w) # dark channel transmission map
    dark_t = 1 - omega*dark_c # corrected dark transmission map
    corrected_t = init_t # initializing corrected transmission map with initial transmission map
    diffch = brightch - darkch # difference between transmission maps

    for i in range(diffch.shape[0]):
        for j in range(diffch.shape[1]):
            if(diffch[i, j] < alpha):
                corrected_t[i, j] = dark_t[i, j] * init_t[i, j]

    return np.abs(corrected_t)

The corrected transmission map is further refined using a guided filter. The guided filter is a spatial domain filter that is used to smooth and enhance structures while preserving edges and fine details. It helps in making the transmission map smoother and more visually pleasing.



In [ ]:
R, G, B = 0, 1, 2

In [ ]:
def boxfilter(I, r):
  M, N = I.shape
  dest = np.zeros((M, N))
  sumY = np.cumsum(I, axis=0)
  dest[:r + 1] = sumY[r:2*r + 1] # top r+1 lines
  dest[r + 1:M - r] = sumY[2*r + 1:] - sumY[:M - 2*r - 1]
  dest[-r:] = np.tile(sumY[-1], (r, 1)) - sumY[M - 2*r - 1:M - r - 1]
  sumX = np.cumsum(dest, axis=1)
  dest[:, :r + 1] = sumX[:, r:2*r + 1] # left r+1 columns
  dest[:, r + 1:N - r] = sumX[:, 2*r + 1:] - sumX[:, :N - 2*r - 1]
  dest[:, -r:] = np.tile(sumX[:, -1][:, None], (1, r)) - sumX[:, N - 2*r - 1:N - r - 1] # right r columns
  return dest


In [ ]:
def guided_filter(I, p, r=15, eps=1e-3):
  M, N = p.shape
  base = boxfilter(np.ones((M, N)), r)
  means = [boxfilter(I[:, :, i], r) / base for i in range(3)]
  mean_p = boxfilter(p, r) / base
  means_IP = [boxfilter(I[:, :, i]*p, r) / base for i in range(3)]
  covIP = [means_IP[i] - means[i]*mean_p for i in range(3)]
  var = defaultdict(dict)
  for i, j in combinations_with_replacement(range(3), 2):
      var[i][j] = boxfilter(I[:, :, i]*I[:, :, j], r) / base - means[i]*means[j]
  a = np.zeros((M, N, 3))
  for y, x in np.ndindex(M, N):
    Sigma = np.array([[var[R][R][y, x], var[R][G][y, x], var[R][B][y, x]],
                          [var[R][G][y, x], var[G][G][y, x], var[G][B][y, x]],
                          [var[R][B][y, x], var[G][B][y, x], var[B][B][y, x]]])
    cov = np.array([c[y, x] for c in covIP])
    a[y, x] = np.dot(cov, inv(Sigma + eps*np.eye(3)))
  b = mean_p - a[:, :, R]*means[R] - a[:, :, G]*means[G] - a[:, :, B]*means[B]
  q = (boxfilter(a[:, :, R], r)*I[:, :, R] + boxfilter(a[:, :, G], r)*I[:, :, G] + boxfilter(a[:, :, B], r)*I[:, :, B] + boxfilter(b, r)) / base

  return q


Generating the final image

In [ ]:
def get_final_image(I, A, refined_t, tmin):
    refined_t_broadcasted = np.broadcast_to(refined_t[:, :, None], (refined_t.shape[0], refined_t.shape[1], 3)) # duplicating the channel of 2D refined map to 3 channels
    J = (I-A) / (np.where(refined_t_broadcasted < tmin, tmin, refined_t_broadcasted)) + A # finding result

    return (J - np.min(J))/(np.max(J) - np.min(J)) # normalized image


In [ ]:
def dehaze(I, tmin=0.1, w=15, alpha=0.4, omega=0.75, p=0.1, eps=1e-3, reduce=False):
    I = np.asarray(im, dtype=np.float64) # Convert the input to a float array.
    I = I[:, :, :3] / 255
    m, n, _ = I.shape
    Idark, Ibright = get_illumination_channel(I, w)
    A = get_atmosphere(I, Ibright, p)

    init_t = get_initial_transmission(A, Ibright)
    if reduce:
        init_t = reduce_init_t(init_t)
    corrected_t = get_corrected_transmission(I, A, Idark, Ibright, init_t, alpha, omega, w)

    normI = (I - I.min()) / (I.max() - I.min())
    refined_t = guided_filter(normI,corrected_t, w, eps) # applying guided filter
    J_refined = get_final_image(I, A, refined_t, tmin)

    enhanced = (J_refined*255).astype(np.uint8)
    f_enhanced = cv2.detailEnhance(enhanced, sigma_s=10, sigma_r=0.15)
    f_enhanced = cv2.edgePreservingFilter(f_enhanced, flags=1, sigma_s=64, sigma_r=0.2)
    return f_enhanced

In [ ]:
def reduce_init_t(init_t):
    init_t = (init_t*255).astype(np.uint8)
    xp = [0, 32, 255]
    fp = [0, 32, 48]
    x = np.arange(256)
    table = np.interp(x, xp, fp).astype('uint8')
    init_t = cv2.LUT(init_t, table)
    init_t = init_t.astype(np.float64)/255
    return init_t

In [ ]:
def reduce_init_t(init_t):
    init_t = (init_t*255).astype(np.uint8)
    xp = [0, 32, 255]
    fp = [0, 32, 48]
    x = np.arange(256)
    table = np.interp(x, xp, fp).astype('uint8')
    init_t = cv2.LUT(init_t, table)
    init_t = init_t.astype(np.float64)/255
    return init_t


End of code

In [ ]:
path="/content/drive/MyDrive/53_og.jpg"
im=cv2.imread(path)
orig=im.copy
tmin = 0.1   # minimum value for t to make J image
w = 15       # window size, which determine the corseness of prior images
alpha = 0.4  # threshold for transmission correction
omega = 0.75 # this is for dark channel prior
p = 0.1      # percentage to consider for atmosphere
eps = 1e-3   # for J image
I = np.asarray(im, dtype=np.float64) # Convert the input to an array.
I = I[:, :, :3] / 255
f_enhanced2 = dehaze(I, tmin, w, alpha, omega, p, eps, True)
details=path.split("/")
cv2.imwrite(f"/content/drive/MyDrive/enhanced_{details[-1]}", f_enhanced2)
  #print(count)
  #count+=1


True

In [ ]:
extract('/content/drive/MyDrive/47.jpg')

In [ ]:
im = cv2.imread('/content/drive/MyDrive/Dark(207)/849.jpg')
orig = im.copy()


In [ ]:
tmin = 0.1   # minimum value for t to make J image
w = 15       # window size, which determine the corseness of prior images
alpha = 0.4  # threshold for transmission correction
omega = 0.75 # this is for dark channel prior
p = 0.1      # percentage to consider for atmosphere
eps = 1e-3   # for J image

In [ ]:
I = np.asarray(im, dtype=np.float64) # Convert the input to an array.
I = I[:, :, :3] / 255

In [ ]:
#f_enhanced = dehaze(I, tmin, w, alpha, omega, p, eps)
f_enhanced2 = dehaze(I, tmin, w, alpha, omega, p, eps, True)



In [ ]:
cv2.imwrite("/content/drive/MyDrive/enhanced_image_2.jpg", f_enhanced2)

True